# Build Stacks

In [ ]:
import subprocess
from pathlib import Path

import geopandas as gpd
import rasterio

## TODO:

1. Transfer condensed stacks to GEE

## User Inputs

In [ ]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = sitename

# AOI tiles
aoi_tiles = f'{proj_dir}/{sitename}_tiles.geojson'

# Resolution
res = 30

# Time
year = 2020
start_date = '2020-05-01'
end_date = '2020-09-30'

# Stack directory
stack_dir = f'{proj_dir}/stacks'

In [ ]:
bands = {
    '1': {
        'name': 'C-VV',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-VV-',
        'suffix': '.vrt'
    },
    '2': {
        'name': 'C-VH',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-VH-',
        'suffix': '.vrt'
    },
    '3': {
        'name': 'C-INC',
        'dir': f'{proj_dir}/Sentinel-1/S1B_{start_date}_{end_date}/vrt',
        'prefix': 'C-INC-',
        'suffix': '.vrt'
    },
    '4': {
        'name': 'L-HH',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-HH-2020-2019-',
        'suffix': '.vrt'
    },
    '5': {
        'name': 'L-HV',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-HV-2020-2019-',
        'suffix': '.vrt'
    },
    '6': {
        'name': 'L-INC',
        'dir': f'{proj_dir}/ALOS-2/mosaic',
        'prefix': 'L-INC-2020-2019-',
        'suffix': '.vrt'
    },
    '7': {
        'name': 'NDVI',
        'dir': f'{proj_dir}/landsat/{year}',
        'prefix': f'landsat_ndvi_{sitename}_{year}_',
        'suffix': '.tif'
    },
    '8': {
        'name': 'TC',
        'dir': f'{proj_dir}/modis/{year}',
        'prefix': f'modis_tc_{sitename}_{year}_',
        'suffix': '.tif'
    },
}

In [ ]:
def build_stack(stack_dir, bands):
    vrt_dir = Path(f'{stack_dir}/vrt')
    if not vrt_dir.exists():
        vrt_dir.mkdir(parents=True)
    gdf_tiles = gpd.read_file(aoi_tiles)
    for i in gdf_tiles.index:
        h = gdf_tiles['h'][i]
        v = gdf_tiles['v'][i]
        m = gdf_tiles['mask'][i]
        g = gdf_tiles['geometry'][i]

        if m == 0:
            continue

        band_data_list = []
        band_name_list = []
        band_num = len(bands.keys())
        for i in range(band_num):
            band = bands[f'{i+1}']
            band_name_list.append(band['name'])
            band_data_list.append(f"{band['dir']}/{band['prefix']}h{h}v{v}{band['suffix']}")

        vrt = vrt_dir / f'{sitename}_stacks_{year}_h{h}v{v}.vrt'
        cmd = (f'gdalbuildvrt -overwrite -separate '
               f'{vrt} {" ".join(band_data_list)}')
        subprocess.check_call(cmd, shell=True)

        with rasterio.open(vrt, 'r+') as dset:
            dset.descriptions = band_name_list

        print(f'Making stack tif for h{h}v{v} ...')
        stack_tif = Path(f'{sitename}_stacks_{year}_h{h}v{v}.tif')
        cmd = (f'gdalwarp '
               f'-overwrite '
               f'-dstnodata -9999 '
               f'-ot Float32 '
               f'-of COG '
               f'-co COMPRESS=LZW '
               f'-co RESAMPLING=NEAREST '
               f'{vrt} {stack_tif}')
        subprocess.check_call(cmd, shell=True)

In [ ]:
build_stack(stack_dir, bands)